In [ ]:
%matplotlib inline
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path 

import dataTools.dictionaries as dcts
from dataTools import tools
from dataTools.data import data_getter
import dataTools.plot.create_figure as cfig 

from dataTools.data.Caribic import Caribic

In [ ]:
# C:\Users\sophie_bauchinger\Documents\GitHub\pyenvs\airvenv\Scripts\activate

# Data objects

Main workflows
* Caribic: Observational data from Caribic2data per pfx per year [GHG, INTtpc - integrated model data over flask sample measurements]
* Phileas / Campaign Data with TPChange interpolation: Get observational and model data from files
* Other campaigns (TPChange model but no obs.): TPChange model + get obs. from SQL or individual approach
* Sonde Data: TPChange interpolation obs + model


In [ ]:
# Test data import from TPChange files 

tpchange_campaigns_obs = {
    'CAR'  : 'CaribicTPChange',
    'SHTR' : 'SouthtracTPChange',
    'WISE' : 'WiseTPChange',
    'ATOM' : 'AtomTPChange',
    'HIPPO': 'HippoTPChange',
    'PGS'  : 'PolstraccTPChange',
    'PHL'  : 'PhileasTPChange',
    'ESM' : 'ESMValTPChange',
    }

# Model data only
tpchange_campaigns_model = {
    'Attrex' : 'MODEL_DATA/ATTREX_TPChange', # N2O available
    'Envisat' : 'MODEL_DATA/ENVISAT_TPChange', # N2O available
    'Euplex' : 'MODEL_DATA/EUPLEX_TPChange', # N2O available
    'SPURT' : 'MODEL_DATA/SPURT_TPChange', # ? Obs data ?
    'TACTS' : 'MODEL_DATA/TACTS_TPChange', # ! Obs data via SQL DB 
    'TC4' : 'MODEL_DATA/TC4_TPChange', # N2O available
    }

In [ ]:
from dataTools.data._global import DataCollection

ID = 'ESM'
pdir = Path('E:/TPChange/' + tpchange_campaigns_obs[ID])
gdf = data_getter.get_TPChange_gdf(pdir)
esm = DataCollection(gdf, ID='ESM')

In [ ]:
car_2013 = Caribic(pfxs = ['INTtpc', 'GHG'], years = [2013], recalculate = True)

# Consistent colors & linestyles 

In [ ]:
# Linestyles per month
fig, ax = plt.subplots()
m_dict = dcts.dict_month()
for m in range(1,13):
    ax.plot(np.arange(1,10), [-m]*9,
            linestyle = '-' if m%2 else '--',
            color = m_dict[f'color_{m}'],
            lw=5,)
ax.set_xlim(0.5,11.5)
ax.legend(handles = cfig.month_legend_handles(), loc='center right');

In [ ]:
# Linestyles per season / Linestyles for typical profile (seasons + av + mean TP)
fig, (ax0, ax1) = plt.subplots(2,1, figsize = (5,6))
s_dict = dcts.dict_season()
for s in range(1,5):
    ax0.plot(np.arange(1,10),
            [-s+0.5*i for i in range(0,9)],
            color = s_dict[f'color_{s}'],
            lw=5,)
ax0.set_xlim(0.5,11.5)
ax0.legend(handles = cfig.season_legend_handles(), loc='lower right');
ax0.xaxis.set_visible(False); ax0.yaxis.set_visible(False)

for s in range(1,5):
    ax1.scatter(np.arange(0,5.5, 0.5),[s]*11,
            color = m_dict[f'color_{s}'],)
ax1.plot(np.arange(0,6), [-1]*6, color='dimgrey', lw = 3)
ax1.plot(np.arange(0,6), [0]*6, color='dimgrey', lw = 3, ls='dashed')
ax1.set_xlim(-0.5,10); ax1.set_ylim(5,-2)
ax1.legend(handles = cfig.typical_profile_legend_handles(), loc='center right')
ax1.xaxis.set_visible(False); ax1.yaxis.set_visible(False)

fig.tight_layout()

In [ ]:
# Linestyles per Tropopause Definition
tps = [
    dcts.get_coord("N2O_residual"),
    dcts.get_coord('int_CARIBIC2_H_rel_TP'),
    dcts.get_coord("int_ERA5_D_HEIGHT_1_5"),
    dcts.get_coord("int_ERA5_D_HEIGHT_2_0"),
    dcts.get_coord("int_ERA5_D_HEIGHT_3_5"),
    dcts.get_coord("int_ERA5_D_HEIGHT_thermTP"),
]; tps.sort(key = lambda x:x.tp_def)

linestyles = dict(
    N2O_residual = 'dashed',
    int_CARIBIC2_H_rel_TP  = '-',
    int_ERA5_D_HEIGHT_1_5 = '-',
    int_ERA5_D_HEIGHT_2_0 = (0, (5,1)),
    int_ERA5_D_HEIGHT_3_5 = (0, (2.5,0.5)),
    int_ERA5_D_HEIGHT_thermTP = (0, (2.5,0.5,1,0.5)),
)

fig, ax = plt.subplots(figsize = (5,2.5))
for i, tp in enumerate(tps):
    ax.plot(np.arange(1,7),[i]*6,
            color = tp.get_color(),
            ls = linestyles[tp.col_name],
            lw=5,)
ax.set_xlim(0.5,11.5); ax.set_ylim(5.5, -0.5)
ax.xaxis.set_visible(False); ax.yaxis.set_visible(False)
ax.legend(handles = cfig.tp_legend_handles(tps, linestyle_dict=linestyles, lw=2), loc='center right');

# Faster programming and general tips and tricks 

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import urllib.request

# with ThreadPoolExecutor() as executor:
#     futures = [executor.submit(
#         wr.joint_station_year_df, id, year) 
#                for id in stations for year in years_per_id[id]]

# Example usage of ThreadPoolExecutor
URLS = ['http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://nonexistent-subdomain.python.org/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# Use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

In [ ]:
import geopandas
from shapely.geometry import Point

def turn_around_XY(df: geodataframe): 
    """ Fix wrongly attributed X / Y coordinates in geometry.  """
    df["true_lat"] = df.geometry.x
    df["true_lon"] = df.geometry.y

    df = df.drop(columns = "geometry")

    geometry = [Point(lon, lat) for lon, lat in zip(df['true_lon'], df['true_lat'])]
    df = df.drop(columns = ["true_lat", "true_lon"])

    gdf = geopandas.GeoDataFrame(data = df, geometry = geometry)
    return gdf